In [ ]:
import redis
r = redis.Redis()
r.mset({'Croatia': 'Zagreb', 'Bahamas': 'Nassau'})
r.get("Bahamas")

In [ ]:
import datetime
today = datetime.date.today().isoformat()
visitors = {'dan', 'jon', 'alex'}
r.sadd(today, *visitors)

In [ ]:
r.smembers(today)

In [ ]:
r.scard(datetime.date.today().isoformat())

In [ ]:
import random

random.seed(444)
hats = {f"hat:{random.getrandbits(32)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}

In [ ]:
hats

In [ ]:
hats

In [ ]:
r = redis.Redis(db=0)

In [ ]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
       pipe.hmset(h_id, hat)
    pipe.execute()

In [ ]:
import websockets
sym = "ethusdt"

In [ ]:
from exchange_feeds.exchange_constants import BINANCE_BASE_WS, BINANCE_TRADE_BLOTTER
url = BINANCE_BASE_WS + sym + BINANCE_TRADE_BLOTTER

In [ ]:
ws = await websockets.connect(url)

In [ ]:
obj = await ws.recv()

In [ ]:
obj

In [ ]:
await ws.close()

In [ ]:
r.hgetall("hat:56854717")

In [ ]:
r.hincrby('hat:56854717', 'quantity', -1)

In [ ]:
r.hgetall("hat:56854717")

In [ ]:
r.hincrby("hat:56854717", 'npurchased', 1)

In [ ]:
r.hgetall("hat:56854717")

In [ ]:
# redis time series practice
import asyncio
import aioredis
from typing import Dict

async def write_to_redis(stream_name: str, fields: Dict[str, str]):
    stream = 'chennai'

    redis = await aioredis.from_url('redis://localhost')
    result = await redis.xadd(stream, fields)
    print(result)
    await redis.close()

stream = 'chennai'
fields = {'temperature': '26', 'humidity': '10'}
#await write_to_redis(stream, fields = fields)

In [ ]:

async def read_from_redis(stream_name: str):
    redis = await aioredis.from_url('redis://localhost')
    result = await redis.xrange(stream_name)
    
    await redis.close()
    return result

stream_name = 'dydx-ethusd-L1'
res1 = await read_from_redis(stream_name)

In [ ]:
res = await redis.xread({stream_name:b'0-0', 'ftx-btcperp-L1': b'0-0'})

In [1]:
from redistoolz import read_all_feeds_from_redis, LOCALHOST, read_feeds_from_redis_synchronous, push_feed_to_postgres, read_feed_from_redis
from exchange_constants import STREAM_NAMES
from time import perf_counter
redis = aioredis.from_url(LOCALHOST, decode_responses=True)

NameError: name 'aioredis' is not defined

In [ ]:

start = perf_counter()
result = await read_feeds_from_redis_synchronous(STREAM_NAMES, redis)
end = perf_counter()
print(f"the program finished in {(end-start):.4f} seconds")    


In [ ]:
from phoboslab.dbhelper import DBReader
db = DBReader()


In [ ]:
start = perf_counter()
result = await read_all_feeds_from_redis(STREAM_NAMES, redis)
end = perf_counter()
print(f"the program finished in {(end-start):.4f} seconds")


In [ ]:
import pandas as pd

In [ ]:
result[4].stream_name

In [ ]:
result[0].get_ids()

In [ ]:
stream_name = result[4].stream_name
await redis.xdel(stream_name, *result[4].get_ids())

In [ ]:
from phoboslab.dbhelper import DBReader
db = DBReader()
#await push_feed_to_postgres(result[0], db=db)

In [ ]:
async def attempt(stream_name, redis):
    feed = await read_feed_from_redis(stream_name, redis)
    await push_feed_to_postgres(feed, db)


In [ ]:
async def main(stream_names, redis):
    tasks = [asyncio.create_task(attempt(stream, redis)) for stream in stream_names]
    await asyncio.gather(*tasks)
    await redis.close()

In [ ]:
result

In [ ]:
stream_names = ['binance-ethusdt-blotter', 'binance-ethusdt-L1']
await main(stream_names, redis)

In [ ]:
feed[0].column_names

In [ ]:
group = 'mygroup'
consumer = 'raj'
streams = {'chennai': '>'}
result = await redis.xreadgroup(group, consumer, streams, count=2)

In [ ]:
result

In [ ]:
await redis.xack('chennai', 'mygroup', '1645027265298')

In [ ]:
async def query_between_time(stream_name: str, start: str, end: str):
    redis = await aioredis.from_url('redis://localhost')
    result = await redis.xrange(name=stream, min=start, max=end)
    print(result)
    await redis.close()


In [ ]:
start_time = '1534866659896'
end_time = '1644333923900'
await query_between_time('chennai', start_time, end_time)

In [ ]:
async def xread_from_redis(stream_name: str):
    redis = await aioredis.from_url('redis://localhost')
    result = await redis.xread(stream_name, block=5000, count=None)
    print(result)
    await redis.close()

stream = 'chennai'
fields = {'temperature': '26', 'humidity': '10'}
#await xread_from_redis({'chennai':'$'})
#await write_to_redis(stream, fields = fields)

In [ ]:
await xread_from_redis({'chennai':'$'})
await write_to_redis(stream, fields = fields)
await xread_from_redis({'chennai':'$'})

In [ ]:
await xread_from_redis({'chennai':'$', 'coimbatore': '$'})


In [ ]:
await read_from_redis('coimbatore')

In [ ]:
write_to_redis(stream, fields = fields)

In [ ]:
async def count():
    print("one")
    await asyncio.sleep(1)
    print("two")

await asyncio.gather(count(), count(), count())


In [ ]:
for i in range(3):
    await test(i)

In [ ]:
loop = asyncio.get_event_loop()
aioredis.from_url('redis://localhost', loop=loop)

In [ ]:
RECORDS = 10
import random
async def add_message_with_sleep(redis, loop, stream):
    records = RECORDS + 1
    for _ in range(records):
        temperature = str(random.randrange(20, 30))
        humidity = str(random.randrange(10, 20))

In [ ]:
temperature = str(random.randrange(20, 30))


In [ ]:
temperature

In [ ]:
import time
RECORDS = 10000
start = time.time()
records = RECORDS + 1
check = []
for _ in range(records):
    temperature = str(random.randrange(20, 30))
    humidity = str(random.randrange(10, 20))
    fields = {
        "temperature": temperature,
        "humidity": humidity,
    }
check.append(fields)
end = time.time()
print(f"finished in {end - start} seconds")

In [ ]:
import websockets
from exchange_feeds.exchange_constants import BINANCE_BASE_WS, BINANCE_TRADE_BLOTTER
sym = 'ethusdt'
url = BINANCE_BASE_WS + sym + BINANCE_TRADE_BLOTTER


In [ ]:
from binance.binance_blotter import BinanceBlotter
bb = BinanceBlotter(url)

In [ ]:
type(bb)

In [ ]:
async def get_message():
    async with websockets.connect(url) as websocket:
        res = await websocket.recv()
        print(res)

In [ ]:
await get_message()

In [ ]:
url

In [ ]:
url

In [ ]:
BINANCE_TRADE_BLOTTER

In [ ]:
os.path.join(BINANCE_BASE_WS, sym, BINANCE_TRADE_BLOTTER)

In [ ]:
from exchange_feeds.exchange_constants import DYDX_BASE_WS, DYDX_SUBSCRIPTION_PAYLOAD
from exchange_feeds.dydx.dydx_orderbook import DYDXOrderBook
import pandas as pd 


In [ ]:
cache = []
async def main():
    symbol = "ETH-USD"
    url = DYDX_BASE_WS
    async with DYDXOrderBook(url, symbol) as book:
        await book.send()
        await book.receive()
        await book.receive()
        for _ in range(5):
            result = await book.receive()
            cache.append(result)
        return result


In [ ]:
result = await main()

In [ ]:
result


In [ ]:
pd.DataFrame(cache).sort_values(by='offset')

In [ ]:
import heapq
from limit_order_book.book import LimitOrderBook

book = LimitOrderBook()



In [ ]:
for item in sorted(cache, key=lambda x: x['offset']):
    await book.update_bids(item)
    await book.update_offers(item)

In [ ]:
await book.best_bid()

In [ ]:
await book.best_ask()

In [ ]:
x = []
x[-1]

In [ ]:
x = []
heapq.heappush(x, ['33', '44', []])

In [ ]:
def get_x():
    return 1


In [ ]:
STREAM_NAMES

In [ ]:
sorted(cache, key=lambda x: x['offset'])

In [ ]:
book.ask_size()

In [ ]:
if book.ask_size():
    print("hmm")

In [ ]:
await book.best_bid()

In [ ]:
await book.best_ask()

In [ ]:
%timeit max(int('8431694293'), int('8431694256'))